In [23]:
import tableauserverclient as TSC
import numpy as np
import pandas as pd
from prep import *

#dispaly jupyter as full width of the screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [28]:
tableau_auth = TSC.TableauAuth(USERNAME, PASSWORD)
server = TSC.Server(SERVER)

with server.auth.sign_in(tableau_auth):
    print(server.version)

2.3


### Data Sources Item

In [10]:
#what is in DataSourceItem
with server.auth.sign_in(tableau_auth):
    #get the top 100 datasources info
    ds, pagination_item = server.datasources.get()
    #to get all datasources info:
    all_info = [ds for ds in TSC.Pager(server.datasources)]
    
    print("\nThere are {} datasources on site: ".format(pagination_item.total_available)) 
    a = ds[0]
         
pd.DataFrame(pd.Series(vars(a)))


There are 1042 datasources on site: 


,0
_certification_note,None
_certified,False
_connections,None
_content_url,MatrixContentViews-Live
_created_at,2015-02-09 19:51:02+00:00
_datasource_type,redshift
_id,b686e88e-01d9-48ff-963f-4573396ff5e3
_initial_tags,{}
_project_id,db7204dc-0252-11e4-9deb-a725883cdf90
_project_name,Sandbox


In [13]:
#DataSourceItem
with server.auth.sign_in(tableau_auth):  
    df_ds = pd.DataFrame(columns = ["name","project_name","datasource_type"])
    
    for n in range(len(ds)):
        i = ds[n]
        server.datasources.populate_connections(i)
        df_ds.loc[n,"name"] = i.name
        df_ds.loc[n,"content_url"] = i._content_url
        df_ds.loc[n,"project_name"] = i.project_name
        df_ds.loc[n,"datasource_type"] =i.datasource_type
        df_ds.loc[n,"_connection_type"] =[connection._connection_type for connection in i.connections]    
        df_ds.loc[n,"embed_password"] = [connection.embed_password for connection in i.connections]    
        df_ds.loc[n,"created_at"] = i._created_at
        df_ds.loc[n,"updated_at"] = i._updated_at
        
    
df_ds.head(2)

,name,project_name,datasource_type,content_url,_connection_type,embed_password,created_at,updated_at
0,(Matrix) Content Views - Live,Sandbox,redshift,MatrixContentViews-Live,[redshift],[None],2015-02-09 19:51:02+00:00,2018-10-10 16:40:05+00:00
1,(Matrix) Content Valuation,Sandbox,redshift,MatrixContentValuation,[redshift],[None],2014-10-23 21:43:42+00:00,2018-10-10 16:39:11+00:00


In [14]:
#what is certification_note and cerified? 
with server.auth.sign_in(tableau_auth):
    tds_cn = [ds for ds in TSC.Pager(server.datasources) if ds.certification_note is not None]
    tds_c = [ds for ds in TSC.Pager(server.datasources) if ds.certified is True]

print (tds_cn, tds_c)

[] []


In [15]:
#ConnectionItem for a DatasourceItem
with server.auth.sign_in(tableau_auth):
    a = server.datasources.get_by_id('52aa5d8e-b3ec-4cfb-b3f2-ce835aa26157')
    
    server.datasources.populate_connections(a)
    c = [vars(connection) for connection in a.connections]
c

[{'_connection_type': 'sqlproxy',
  '_datasource_id': None,
  '_datasource_name': None,
  '_id': 'aab8d6dd-c0d7-4f1c-bdcf-dbc79e45809e',
  'connection_credentials': None,
  'embed_password': None,
  'password': None,
  'server_address': 'i-0e5362952d191',
  'server_port': '80',
  'username': ''}]

### WorkbookItem

In [16]:
with server.auth.sign_in(tableau_auth):
    all_wb, pagination_item = server.workbooks.get()
    print("\nThere are {} workbook on site: ".format(pagination_item.total_available))        
    
    all_info = [wb for wb in TSC.Pager(server.workbooks) if wb.name.startswith('Alert')]
    
    
    df_wb = pd.DataFrame(columns = ["name","project_name"])
    
    for n in range(len(all_info)):
        i = all_info[n]
        #populate the connection and views
        server.workbooks.populate_connections(i)
        server.workbooks.populate_views(i)
        
        views =[vars(view) for view in i.views]
        
        df_wb.loc[n,"name"] = i.name
        df_wb.loc[n,"content_url"] = i._content_url
        df_wb.loc[n,"project_name"] = i.project_name
        df_wb.loc[n,"created_at"] = i._created_at
        df_wb.loc[n,"updated_at"] = i._updated_at
        df_wb.loc[n,"#_views"] = len(views)
        df_wb.loc[n,"_show_tabs"] = i._show_tabs
        df_wb.loc[n,"owner_id"] = i.owner_id
        df_wb.loc[n,"size"] = i.size
        
        try:
            df_wb.loc[n,"_connection_type"] =[connection._connection_type for connection in i.connections]    
            df_wb.loc[n,"embed_password"] = [connection.embed_password for connection in i.connections]
            df_wb.loc[n,"datasource_name"] = [connection.datasource_name for connection in i.connections]
        except: 
            pass
        
    
df_wb.head(2)


There are 1414 workbook on site: 


,name,project_name,content_url,created_at,updated_at,#_views,_show_tabs,owner_id,size,_connection_type,embed_password,datasource_name
0,Alert_RoyalityEvent,Data Governance & Quality,Alert_RoyalityEvent,2018-08-22 22:23:26+00:00,2019-02-01 21:06:59+00:00,4.0,True,7f032f37-05c7-4093-8ec8-ac457b00a488,1.0,[sqlproxy],[None],[Alert - Royalty Events]
1,Alert - Subscription Cancel and Cancel rate,Data Governance & Quality,Alert-SubscriptionCancelandCancelrate,2018-09-05 01:06:37+00:00,2019-01-31 19:38:49+00:00,3.0,True,7f032f37-05c7-4093-8ec8-ac457b00a488,1.0,"[sqlproxy, sqlproxy]","[None, None]","[(Matrix) Subscriber Activity - 2 yr, Alert-Su..."


In [30]:
#for individual workbook
with server.auth.sign_in(tableau_auth):
    all_info = [wb for wb in TSC.Pager(server.workbooks) if wb.name.startswith('Alert')]
    wb = all_info[0]
    #populate the connection
    server.workbooks.populate_connections(wb)
    server.workbooks.populate_views(wb)
    c = [[connection.id, connection.connection_type, connection.datasource_name] for connection in wb.connections]
    c_info = [vars(connection) for connection in wb.connections]
    views =[vars(view) for view in wb.views]

ds_name = [pd.Series([c[0][0]], name = 'connection_id'),
           pd.Series([c[0][1]], name = 'connection_type'),
           pd.Series([c[0][2]], name = 'datasource_name'),
           pd.Series(len(views), name = '#_views')]
pd.DataFrame(pd.Series(vars(wb))).append(ds_name).drop(index=['_connections','_views'])


,0
_content_url,Alert_RoyalityEvent
_created_at,2018-08-22 22:23:26+00:00
_id,9a75a75a-bcc3-4371-ba9d-386da039bbb4
_initial_tags,{}
_preview_image,None
_project_id,ffec875f-cdfe-42b9-8f91-2a994d2bc07f
_project_name,Data Governance & Quality
_show_tabs,True
_size,1
_updated_at,2019-02-01 21:06:59+00:00


In [18]:
c_info

[{'_connection_type': 'sqlproxy',
  '_datasource_id': '52aa5d8e-b3ec-4cfb-b3f2-ce835aa26157',
  '_datasource_name': 'Alert - Royalty Events',
  '_id': 'aab8d6dd-c0d7-4f1c-bdcf-dbc79e45809e',
  'connection_credentials': None,
  'embed_password': None,
  'password': None,
  'server_address': 'i-0e5362952d191',
  'server_port': '80',
  'username': ''}]

In [20]:
pd.DataFrame(views).head(2)

,_content_url,_csv,_id,_image,_initial_tags,_name,_owner_id,_pdf,_preview_image,_project_id,_total_views,_workbook_id,tags
0,Alert_RoyalityEvent/sheets/RecordText,None,c31004cc-7d18-4e01-a027-3536e89cf1da,None,{},Royalty-ContentViewRecordText,None,None,None,None,None,9a75a75a-bcc3-4371-ba9d-386da039bbb4,{}
1,Alert_RoyalityEvent/sheets/RecordText-YoY,None,ce1bf347-7928-4c40-895d-2c13bfc5fb71,None,{},RecordText-YoY,None,None,None,None,None,9a75a75a-bcc3-4371-ba9d-386da039bbb4,{}


### Permision/Version Issues

In [ ]:
#groups
with server.auth.sign_in(tableau_auth):
    all_groups, pagination_item = server.groups.get()
    g = all_groups[1]
    
pd.DataFrame(pd.Series(vars(g)))

In [ ]:
#users
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get()
    u = all_users[1]
    
pd.DataFrame(pd.Series(vars(u)))

In [ ]:
#sites
with server.auth.sign_in(tableau_auth):
    for s in TSC.Pager(server.sites):
        print(s.content_url)

In [ ]:
#tags
with server.auth.sign_in(tableau_auth):
    sample_view = wb.views[0]
    original_tag_set = set(sample_view.tags)
    sample_view.tags.add("view_tag")
    server.views.update(sample_view)
    print("\nView's old tag set: {}".format(original_tag_set))
    print("View's new tag set: {}".format(sample_view.tags))

In [ ]:
#Previous Image was introduced in Version 2.5
with server.auth.sign_in(tableau_auth):
        # Step 2: Query for the view that we want an image of
        all_views, pagination_item = server.views.get()
        view_item = all_views[0]

        # Step 3: Query the image endpoint and save the image to the specified location
        image_req_option = TSC.ImageRequestOptions(imageresolution=TSC.ImageRequestOptions.Resolution.High)
        server.views.populate_image(view_item, image_req_option)

        with open('./a.jpg', "wb") as image_file:
            image_file.write(view_item.image)

        print("View image saved to ./a.jpg'")

In [39]:
with server.auth.sign_in(tableau_auth):
    all_schedules, pagination_item = server.schedules.get()
    print("\nThere are {} schedules on site: ".format(pagination_item.total_available))

all_schedules[0]


There are 37 schedules on site: 


<Schedule#ee4745b0-bcea-4de2-87df-5623176858ca "Weekday early mornings (4am)" None>

### VIEWS

In [22]:
with server.auth.sign_in(tableau_auth):
    all_views, pagination_item = server.views.get()
    print("\nThere are {} views on site: ".format(pagination_item.total_available))        
    v = all_views[0]
    
pd.DataFrame(pd.Series(vars(v)))


There are 7282 views on site: 


,0
_content_url,DataDictionary_1/sheets/DataDictionary
_csv,None
_id,dbab3eaa-0252-11e4-a8fc-b760e4643919
_image,None
_initial_tags,{}
_name,Data Dictionary
_owner_id,9deb418f-b79c-4d77-ae25-3b07b17a5a5a
_pdf,None
_preview_image,None
_project_id,None


### SubscriptionsItem

In [7]:
with server.auth.sign_in(tableau_auth):
    #target = ('c7a9327e-1cda-4504-b026-ddb43b976d1d', 'workbook')
    all_sub = server.subscriptions.get()
    sub = all_sub[1]
vars(sub)

{'_page_number': 1, '_page_size': 100, '_total_available': 0}

### ProjectItem

In [10]:
with server.auth.sign_in(tableau_auth): 
        # get all projects on site
        all_projects, pagination_item = server.projects.get()
        pj = all_projects[0]
        #print([proj.name for proj in all_projects])

pd.DataFrame(pd.Series(vars(pj)))

,0
_content_permissions,ManagedByOwner
_id,db7204dc-0252-11e4-abfc-ef3c97791691
_name,Marketing
description,Reports and Dashboards for the Marketing team....
parent_id,None


### Get specific items with id or criteria

In [ ]:
workbook = server.workbooks.get_by_id('9a75a75a-bcc3-4371-ba9d-386da039bbb4')
all_info = [wb for wb in TSC.Pager(server.workbooks) if wb.name.startswith('Alert')]